In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [30]:
data=pd.read_csv("insurance.csv")
n_count=data.isnull().sum()
print(n_count)


age         0
sex         0
bmi         2
children    0
smoker      0
region      0
expenses    0
dtype: int64


In [37]:
data['bmi']=data.fillna(data['bmi'].mean)
n_count=data.isnull().sum()
print(n_count[n_count>0])
data

Series([], dtype: int64)


,age,sex,bmi,children,smoker,region,expenses
0,19,female,19,0,yes,southwest,16884.92
1,18,male,18,1,no,southeast,1725.55
2,28,male,28,3,no,southeast,4449.46
3,33,male,33,0,no,northwest,21984.47
4,32,male,32,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,50,3,no,northwest,10600.55
1334,18,female,18,0,no,northeast,2205.98
1335,18,female,18,0,no,southeast,1629.83
1336,21,female,21,0,no,southwest,2007.95


In [130]:
gender=data.iloc[:,1:2].values
le=LabelEncoder()
gender=le.fit_transform(gender)
gender=pd.DataFrame(gender)
gender.columns=['gender']
le_gender_mapping=dict(zip(le.classes_,le.transform(le.classes_)))
print(le_gender_mapping)


{'female': 0, 'male': 1}


C:\Users\RUDRA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [129]:

smokers=data.iloc[:,4:5].values

le=LabelEncoder()
smokers=le.fit_transform(smokers)
smokers=pd.DataFrame(smokers)
smokers.columns=['smokers']
le_smokers_mapping=dict(zip(le.classes_,le.transform(le.classes_)))
print(le_smokers_mapping)
print(smokers)

{'no': 0, 'yes': 1}
      smokers
0           1
1           0
2           0
3           0
4           0
...       ...
1333        0
1334        0
1335        0
1336        0
1337        1

[1338 rows x 1 columns]


C:\Users\RUDRA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [133]:
regions=data.iloc[:,5:6]
ohe=OneHotEncoder()
regions=ohe.fit_transform(regions).toarray()
regions=pd.DataFrame(regions)
regions.columns=["northeast","northwest","southeast","southwest"]



In [139]:
X_num=data[["age","bmi","children"]]
X_final=pd.concat([X_num,gender,smokers,regions],axis=1)
y_final=data[["expenses"]].copy()


In [140]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=0)

In [167]:
pipeline_lr=Pipeline([('s_scaler',StandardScaler()),
                     ('lr',LinearRegression())])

In [168]:
pipeline_svr=Pipeline([('s_scaler',StandardScaler()),
                  ('svr',SVR(kernel='linear',C=300))])

In [169]:
pipeline_dr=Pipeline([('s_scaler',StandardScaler()),
                     ('dr',DecisionTreeRegressor(random_state=0))])

In [170]:
pipeline_rf=Pipeline([('s_scaler',StandardScaler()),
                     ('rf',RandomForestRegressor(n_estimators=100,criterion='mse',random_state=1,n_jobs=-1))])

In [174]:
pipelines=[pipeline_lr,pipeline_dr,pipeline_svr,pipeline_rf]


In [175]:
best_accuracy=0.0
best_classifier=0
best_pipeline=""
pipe_dict={0:'Linear Regression',1:'Support Vector Regression',2:'Decision Tree Regressor',3:'Random Forest Regressor'}


In [177]:
for pipe in pipelines:
    pipe.fit(X_train,y_train)

C:\Users\RUDRA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\RUDRA\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [178]:
for i,model in enumerate (pipelines):
    print(" {} Test Accuracy :{}".format(pipe_dict[i],model.score(X_test,y_test)))
    

 Linear Regression Test Accuracy :0.7589469619031977
 Support Vector Regression Test Accuracy :0.5330268777880203
 Decision Tree Regressor Test Accuracy :0.6202006129141526
 Random Forest Regressor Test Accuracy :0.6888569273225582


In [179]:
for i,model in enumerate (pipelines):
    if model.score(X_test,y_test)>best_accuracy:
        best_accuracy=model.score(X_test,y_test)
        best_pipeline=model
        best_classifier=i

print("Best Model Classifier :{} with Accuracy:{}".format(pipe_dict[best_classifier],best_accuracy))        

Best Model Classifier :Linear Regression with Accuracy:0.7589469619031977
